In [3]:
import spacy
from fastprogress import *
nlp = spacy.load('en_core_web_sm')

In [36]:
from tqdm import tqdm_notebook, tqdm

In [22]:
from concurrent.futures import ProcessPoolExecutor
import re

def parallel(func, arr, max_workers=4):
    if max_workers<2: results = list(progress_bar(map(func, enumerate(arr)), total=len(arr)))
    else:
        with ProcessPoolExecutor(max_workers=max_workers) as ex:
            return list(progress_bar(ex.map(func, enumerate(arr)), total=len(arr)))
    if any([o is not None for o in results]): return results
    
def listify(o):
    if o is None: return []
    if isinstance(o, list): return o
    if isinstance(o, str): return [o]
    if isinstance(o, Iterable): return list(o)
    return [o]

def compose(x, funcs, *args, order_key='_order', **kwargs):
    key = lambda o: getattr(o, order_key, 0)
    for f in sorted(listify(funcs), key=key): x = f(x, **kwargs)
    return x


In [20]:
class TokenizeProcessor():
    def __init__(self, nlp, chunksize=2000, pre_rules=None, post_rules=None, max_workers=4): 
        self.chunksize,self.max_workers = chunksize,max_workers
        self.tokenizer = nlp.tokenizer
        self.pre_rules  = pre_rules 
        self.post_rules = post_rules

    def proc_chunk(self, args):
        i,chunk = args
        chunk = [compose(t, self.pre_rules) for t in chunk]
        docs = [[d.text for d in doc] for doc in self.tokenizer.pipe(chunk)]
        docs = [compose(t, self.post_rules) for t in docs]
        return docs

    def __call__(self, items): 
        toks = []
        chunks = [items[i: i+self.chunksize] for i in (range(0, len(items), self.chunksize))]
        toks = parallel(self.proc_chunk, chunks, max_workers=self.max_workers)
        return sum(toks, [])
    
    def proc1(self, item): return self.proc_chunk([item])[0]
    
    def deprocess(self, toks): return [self.deproc1(tok) for tok in toks]
    def deproc1(self, tok):    return " ".join(tok)

In [50]:
def multi_thread_based_tokenizations(nlp, text_list, n_threads=4):
    docs = nlp.pipe(text_list, n_threads = n_threads)
    word_sequences = []
    
    for doc in tqdm(docs):
        word_seq = []
        for token in doc:
            word_seq.append(token.text)
        word_sequences.append(word_seq)
    return word_sequences

In [9]:
texts = ["This is a text", "These are lots of texts"]

In [24]:
tp = TokenizeProcessor(nlp)

In [25]:
tp(texts)

[['This', 'is', 'a', 'text'], ['These', 'are', 'lots', 'of', 'texts']]

In [43]:
word_sequences = multi_thread_based_tokenizations(nlp, texts)
word_sequences

2it [00:00, 101.57it/s]


[['This', 'is', 'a', 'text'], ['These', 'are', 'lots', 'of', 'texts']]

### Example Test

In [26]:
import pandas as pd
df = pd.read_csv('/Users/ankur.kumar/Desktop/Work/projects/client/pantaloon/Pantaloon-MH-processed.csv')[['UID', 'Cleaned_Verbatim']]

In [44]:
df = df.drop_duplicates(['UID'])

In [45]:
df = df[0:10000]
df.shape

(10000, 2)

In [46]:
df['Cleaned_Verbatim'] = df['Cleaned_Verbatim'].astype(str)

In [47]:
Verbatims = df.Cleaned_Verbatim.values.tolist()

In [54]:
%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, 1)
len(word_sequences)

0it [00:00, ?it/s]

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 5.01 µs


10000it [00:42, 237.23it/s]


10000

In [52]:
%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, 2)
len(word_sequences)

0it [00:00, ?it/s]

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 6.2 µs


100it [00:00, 273.06it/s]


100

In [53]:
%time
word_sequences = multi_thread_based_tokenizations(nlp, Verbatims, 3)
len(word_sequences)

0it [00:00, ?it/s]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.25 µs


100it [00:00, 275.80it/s]


100